<a href="https://colab.research.google.com/github/santiagonajera/OptimizacionRutas/blob/main/OptimoTransporte_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
from scipy.optimize import linprog

# Definimos los costos de envío (en forma de vector)
# Orden: CD1->C1, CD1->C2, CD1->C3,
#        CD2->C1, CD2->C2, CD2->C3,
#        CD3->C1, CD3->C2, CD3->C3
costos = np.array([
    [5, 4, 3],
    [8, 4, 3],
    [9, 7, 5]
]).flatten()

# Vector objetivo (minimizar costo total)
c = costos

# Variables de decisión: x_ij = cantidad enviada desde CD i a Ciudad j
# Hay 3x3 = 9 variables
# x[0] = CD1 -> C1, x[1] = CD1 -> C2, x[2] = CD1 -> C3
# x[3] = CD2 -> C1, x[4] = CD2 -> C2, x[5] = CD2 -> C3
# x[6] = CD3 -> C1, x[7] = CD3 -> C2, x[8] = CD3 -> C3

# Restricciones de oferta (sumatoria por fila <= oferta)
A_ub = []
b_ub = []

# Oferta de cada centro de distribución
oferta = np.array([200, 400, 300])
for i in range(3):
    row = np.zeros(9)
    for j in range(3):
        row[i * 3 + j] = 1  # Variable x_ij
    A_ub.append(row)
    b_ub.append(oferta[i])

# Demanda de cada ciudad (sumatoria por columna >= demanda)
demanda = np.array([500, 200, 200])
for j in range(3):
    row = np.zeros(9)
    for i in range(3):
        row[i * 3 + j] = 1  # Variable x_ij
    A_ub.append(row)
    b_ub.append(demanda[j])

# Ahora, como linprog maneja desigualdades <=, convertimos las demandas >= a <=
# Pero como no se puede usar >= directamente, usamos:
# Para demanda: sum(x_i,j) >= demanda_j → -sum(x_i,j) <= -demanda_j
# Entonces agregamos restricciones negativas
for j in range(3):
    row = np.zeros(9)
    for i in range(3):
        row[i * 3 + j] = -1
    A_ub.append(row)
    b_ub.append(-demanda[j])

# También debemos asegurar que todas las variables sean no negativas
bounds = [(0, None) for _ in range(9)]

# Resolver el problema
result = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')

# Mostrar resultados
if result.success:
    print("✅ Problema resuelto exitosamente.")
    print(f"Costo mínimo total: ${result.fun:.2f}")

    # Reorganizar soluciones en matriz
    x = result.x.reshape((3, 3))
    print("\nCantidad enviada (CD -> Ciudad):")
    print("           Ciudad 1     Ciudad 2     Ciudad 3")
    print("CD 1      ", x[0,0], "         ", x[0,1], "         ", x[0,2])
    print("CD 2      ", x[1,0], "         ", x[1,1], "         ", x[1,2])
    print("CD 3      ", x[2,0], "         ", x[2,1], "         ", x[2,2])
else:
    print("❌ No se pudo encontrar una solución factible.")

✅ Problema resuelto exitosamente.
Costo mínimo total: $5100.00

Cantidad enviada (CD -> Ciudad):
           Ciudad 1     Ciudad 2     Ciudad 3
CD 1       200.0           0.0           0.0
CD 2       -0.0           200.0           200.0
CD 3       300.0           0.0           0.0


In [4]:
import numpy as np
from scipy.optimize import linprog

# -----------------------------
# Datos de entrada (pueden cargarse desde archivos o entradas)
# -----------------------------

# Matriz de costos: [centro][ciudad]
costos = np.array([
    [5.00, 4.00, 3.00],  # CD1 → C1, C2, C3
    [8.00, 4.00, 3.00],  # CD2 → C1, C2, C3
    [9.00, 7.00, 5.00]   # CD3 → C1, C2, C3
])

# Demanda por ciudad (vector)
demanda = np.array([500, 200, 200])  # Ciudad1, Ciudad2, Ciudad3

# Oferta por centro de distribución (vector)
oferta = np.array([200, 400, 300])  # CD1, CD2, CD3

# -----------------------------
# Parámetros del problema
# -----------------------------
num_centros = len(oferta)
num_ciudades = len(demanda)

# Verificación de dimensiones
if costos.shape != (num_centros, num_ciudades):
    raise ValueError("La matriz de costos debe tener dimensiones (num_centros, num_ciudades)")

# -----------------------------
# Preparación del modelo de optimización
# -----------------------------

# Vector objetivo: costo total
c = costos.flatten()

# Restricciones de igualdad: oferta y demanda
A_eq = []
b_eq = []

# 1. Restricciones de oferta: suma de envíos desde cada centro = oferta
for i in range(num_centros):
    row = np.zeros(num_centros * num_ciudades)
    for j in range(num_ciudades):
        row[i * num_ciudades + j] = 1  # x[i,j]
    A_eq.append(row)
    b_eq.append(oferta[i])

# 2. Restricciones de demanda: suma de envíos a cada ciudad = demanda
for j in range(num_ciudades):
    row = np.zeros(num_centros * num_ciudades)
    for i in range(num_centros):
        row[i * num_ciudades + j] = 1  # x[i,j]
    A_eq.append(row)
    b_eq.append(demanda[j])

# Límites inferiores: todas las variables >= 0
bounds = [(0, None) for _ in range(num_centros * num_ciudades)]

# Resolver el problema de transporte
result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

# Mostrar resultados
if result.success:
    print("✅ Problema resuelto exitosamente.")
    print(f"Costo mínimo total: ${result.fun:.2f}")

    # Reorganizar la solución en una matriz
    x = result.x.reshape((num_centros, num_ciudades))

    print("\nCantidad enviada (Centro → Ciudad):")
    print("           ", end="")
    for j in range(num_ciudades):
        print(f"Ciudad{j+1:>8}", end="")
    print()
    print("-" * 40)

    for i in range(num_centros):
        print(f"CD{i+1:<8}", end="")
        for j in range(num_ciudades):
            print(f"{x[i,j]:>10.0f}", end="")
        print()
else:
    print("❌ No se pudo encontrar una solución factible.")

✅ Problema resuelto exitosamente.
Costo mínimo total: $5100.00

Cantidad enviada (Centro → Ciudad):
           Ciudad       1Ciudad       2Ciudad       3
----------------------------------------
CD1              200         0         0
CD2                0       200       200
CD3              300         0        -0


In [5]:
import numpy as np
from scipy.optimize import linprog
import pandas as pd

# -----------------------------
# Cargar datos desde Excel en línea
# -----------------------------
url = "https://github.com/santiagonajera/OptimizacionRutas/raw/refs/heads/main/OptimoTransportes.xlsx"

# Leer las hojas del archivo Excel
try:
    # Leer hoja 'COSTOS'
    df_costos = pd.read_excel(url, sheet_name='COSTOS', index_col=0)
    costos = df_costos.values  # Convertir a NumPy array

    # Leer hoja 'DEMANDA'
    df_demanda = pd.read_excel(url, sheet_name='DEMANDA')
    demanda = df_demanda['DEMANDA'].values  # Extraer columna DEMANDA

    # Leer hoja 'OFERTA'
    df_oferta = pd.read_excel(url, sheet_name='OFERTA')
    oferta = df_oferta['OFERTA'].values  # Extraer columna OFERTA

    print("✅ Datos cargados correctamente desde el archivo Excel.")

except Exception as e:
    print(f"❌ Error al cargar el archivo Excel: {e}")
    exit()

# -----------------------------
# Parámetros del problema
# -----------------------------
num_centros = len(oferta)
num_ciudades = len(demanda)

# Verificación de dimensiones
if costos.shape != (num_centros, num_ciudades):
    raise ValueError(f"La matriz de costos debe tener tamaño ({num_centros}, {num_ciudades}), pero tiene {costos.shape}")

# -----------------------------
# Preparación del modelo de optimización
# -----------------------------

# Vector objetivo: costo total
c = costos.flatten()

# Restricciones de igualdad: oferta y demanda
A_eq = []
b_eq = []

# 1. Restricciones de oferta: suma de envíos desde cada centro = oferta
for i in range(num_centros):
    row = np.zeros(num_centros * num_ciudades)
    for j in range(num_ciudades):
        row[i * num_ciudades + j] = 1  # x[i,j]
    A_eq.append(row)
    b_eq.append(oferta[i])

# 2. Restricciones de demanda: suma de envíos a cada ciudad = demanda
for j in range(num_ciudades):
    row = np.zeros(num_centros * num_ciudades)
    for i in range(num_centros):
        row[i * num_ciudades + j] = 1  # x[i,j]
    A_eq.append(row)
    b_eq.append(demanda[j])

# Límites inferiores: todas las variables >= 0
bounds = [(0, None) for _ in range(num_centros * num_ciudades)]

# Resolver el problema de transporte
result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

# Mostrar resultados
if result.success:
    print("\n✅ Problema resuelto exitosamente.")
    print(f"Costo mínimo total: ${result.fun:.2f}")

    # Reorganizar la solución en una matriz
    x = result.x.reshape((num_centros, num_ciudades))

    print("\nCantidad enviada (Centro → Ciudad):")
    print("           ", end="")
    for j in range(num_ciudades):
        print(f"Ciudad{j+1:>8}", end="")
    print()
    print("-" * 40)

    for i in range(num_centros):
        print(f"CD{i+1:<8}", end="")
        for j in range(num_ciudades):
            print(f"{x[i,j]:>10.0f}", end="")
        print()
else:
    print("\n❌ No se pudo encontrar una solución factible.")
    print(result.message)

✅ Datos cargados correctamente desde el archivo Excel.

✅ Problema resuelto exitosamente.
Costo mínimo total: $5100.00

Cantidad enviada (Centro → Ciudad):
           Ciudad       1Ciudad       2Ciudad       3
----------------------------------------
CD1              200         0         0
CD2                0       200       200
CD3              300         0        -0
